In [1]:
import matchzoo as mz
import pandas as pd
prefix_suffix_pairs_background = pd.read_csv("../data/prefix_suffix_pairs.txt", header=None)


Using TensorFlow backend.
Generating grammar tables from /usr/lib/python3.7/lib2to3/Grammar.txt
Generating grammar tables from /usr/lib/python3.7/lib2to3/PatternGrammar.txt


In [3]:
prefix_small = prefix_suffix_pairs_background.iloc[1:10000,:]

In [4]:
import numpy as np
type(prefix_suffix_pairs_background)
prefix_small.columns = ["text_left", "text_right"]
train_set = mz.pack(prefix_small)
data = pd.DataFrame({
    'text_left': list(prefix_small["text_left"].apply(str)),
    'text_right': list(prefix_small["text_right"].apply(str)),
    'label': 1
})
my_pack = mz.pack(data)


In [8]:
model = mz.models.CDSSM()
model.params['name'] = 'Prefix-suffix CDSSM'
model.params['task'] = 'rankingtask'
model.params['filters'] = 300
model.params['kernel_size'] = 3
model.params['strides'] = 1
model.params['padding'] = 'same'
model.params['conv_activation_func'] = 'tanh'
model.params['w_initializer'] = 'glorot_normal'
model.params['b_initializer'] = 'zeros'
model.params['mlp_num_layers'] = 1
model.params['mlp_num_units'] = 300
model.params['mlp_num_fan_out'] = 128
model.params['mlp_activation_func'] = 'tanh'
model.params['dropout_rate'] = 0.8
model.params['optimizer'] = 'adadelta'
model.guess_and_fill_missing_params()
model.build()
model.compile()
model.backend.summary()

Parameter "input_shapes" set to [(10, 30), (10, 30)].


ValueError: Invalid task type.

In [10]:
model_ok, train_ok, preprocesor_ok = mz.auto.prepare(
    model=mz.models.DSSM(),
    data_pack=my_pack
)
# test_ok = preprocesor_ok.transform(test_data_pack, verbose=0)
model_ok.fit(*train_ok.unpack(), batch_size=32)
# model_ok.evaluate(*test_ok.unpack())



Processing text_left with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit => StopRemovalUnit => NgramLetterUnit: 100%|██████████| 5609/5609 [00:00<00:00, 11845.35it/s]
Processing text_right with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit => StopRemovalUnit => NgramLetterUnit: 100%|██████████| 5407/5407 [00:00<00:00, 12024.98it/s]
Building VocabularyUnit from a datapack.: 100%|██████████| 132345/132345 [00:00<00:00, 3254850.14it/s]
Processing text_left with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit => StopRemovalUnit => NgramLetterUnit => WordHashingUnit: 100%|██████████| 5609/5609 [00:00<00:00, 9159.04it/s]
Processing text_right with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit => StopRemovalUnit => NgramLetterUnit => WordHashingUnit: 100%|██████████| 5407/5407 [00:00<00:00, 8807.48it/s]


Parameter "name" set to DSSM.
Parameter "mlp_num_layers" set to 3.
Parameter "mlp_num_units" set to 64.
Parameter "mlp_num_fan_out" set to 32.
Parameter "mlp_activation_func" set to relu.
Epoch 1/1
9999/9999 [==============================] - 5s 464us/step - loss: 0.0026


In [13]:
method_list = [func for func in dir(model_ok) if callable(getattr(model_ok, func))]
method_list

['__class__',
 '__delattr__',
 '__dir__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '_backend',
 '_eval_metric_on_data_frame',
 '_make_embedding_layer',
 '_make_inputs',
 '_make_multi_layer_perceptron_layer',
 '_make_output_layer',
 '_remap_keras_metric',
 '_separate_metrics',
 '_set_param_default',
 'backend',
 'build',
 'compile',
 'evaluate',
 'fit',
 'fit_generator',
 'get_default_params',
 'get_default_preprocessor',
 'guess_and_fill_missing_params',
 'load_embedding_matrix',
 'predict',
 'save']

In [14]:
model_ok.predict("hallo ")

TypeError: predict() missing 1 required positional argument: 'x'